In [ ]:
# Step 1: Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Step 2: Create synthetic dataset
np.random.seed(42)
n_students = 200  # number of students

data = {
    'school': np.random.choice(['GP', 'MS'], n_students),
    'sex': np.random.choice(['F', 'M'], n_students),
    'age': np.random.randint(15, 22, n_students),
    'studytime': np.random.randint(1, 5, n_students),
    'failures': np.random.randint(0, 4, n_students),
    'absences': np.random.randint(0, 20, n_students),
    'G1': np.random.randint(0, 21, n_students),
    'G2': np.random.randint(0, 21, n_students),
    'G3': np.random.randint(0, 21, n_students)
}

df = pd.DataFrame(data)

# Step 3: Save dataset to CSV (optional, for reference)
df.to_csv("synthetic_student_data.csv", index=False)
print("Synthetic CSV created successfully!\n")
print(df.head())

# Step 4: Handle missing values (none in synthetic, but good practice)
df = df.fillna(method='ffill')

# Step 5: Encode categorical variables
le = LabelEncoder()
for col in df.select_dtypes(include='object').columns:
    df[col] = le.fit_transform(df[col])

# Step 6: Split data into features and labels
X = df.drop('G3', axis=1)  # Features
y = df['G3']               # Target: Final grade

# Step 7: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("\nData preprocessing completed successfully!")
print("Training samples:", X_train.shape[0])
print("Testing samples:", X_test.shape[0])


Synthetic CSV created successfully!

  school sex  age  studytime  failures  absences  G1  G2  G3
0     GP   F   18          1         2        14  18   6   0
1     MS   M   17          3         0        16   1  17  10
2     GP   F   15          2         3        10  14   1   7
3     GP   F   18          4         0        16  10  16   9
4     GP   M   20          2         2        12   7  10   6

Data preprocessing completed successfully!
Training samples: 160
Testing samples: 40


/tmp/ipython-input-281810512.py:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')


In [ ]:
# Module 2: AI Model Development

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# Step 1: Initialize model
model = DecisionTreeRegressor(random_state=42)

# Step 2: Train model
model.fit(X_train, y_train)

# Step 3: Test model
y_pred = model.predict(X_test)

# Step 4: Evaluate performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model trained successfully!")
print(f"Mean Squared Error: {mse:.2f}")
print(f"R2 Score: {r2:.2f}")

# Step 5: Save trained model
joblib.dump(model, "student_model.pkl")
print("Model saved as student_model.pkl")


Model trained successfully!
Mean Squared Error: 63.15
R2 Score: -0.68
Model saved as student_model.pkl


In [ ]:
# Module 3: Adaptive Recommendation System

# Step 1: Define student learning levels
def categorize_student(grade):
    if grade >= 16:
        return "Advanced"
    elif grade >= 11:
        return "Intermediate"
    else:
        return "Beginner"

# Step 2: Content recommendation rules
def recommend_content(level):
    recommendations = {
        "Beginner": ["Basic Math Exercises", "Simple Programming Tasks"],
        "Intermediate": ["Project Work", "Medium-level Quizzes"],
        "Advanced": ["Advanced Projects", "Competitive Coding Challenges"]
    }
    return recommendations[level]

# Step 3: Test with sample grades
sample_grades = [5, 12, 18]
for grade in sample_grades:
    level = categorize_student(grade)
    content = recommend_content(level)
    print(f"Grade: {grade}, Level: {level}, Recommended Content: {content}")


Grade: 5, Level: Beginner, Recommended Content: ['Basic Math Exercises', 'Simple Programming Tasks']
Grade: 12, Level: Intermediate, Recommended Content: ['Project Work', 'Medium-level Quizzes']
Grade: 18, Level: Advanced, Recommended Content: ['Advanced Projects', 'Competitive Coding Challenges']


In [ ]:
!pip install flask-ngrok --quiet

from flask import Flask, request, render_template_string
from flask_ngrok import run_with_ngrok  # <- correct syntax with dot, not underscore
import pandas as pd
import joblib

# Load trained model
model = joblib.load("student_model.pkl")

# Initialize Flask app
app = Flask(__name__)
run_with_ngrok(app)  # Creates public URL for Colab


# HTML template for input
html_template = """
<!DOCTYPE html>
<html>
<head><title>Student Performance Prediction</title></head>
<body>
    <h2>Enter Student Details</h2>
    <form action="/" method="post">
        StudyTime (1-4): <input type="number" name="studytime" step="1"><br>
        Failures (0-3): <input type="number" name="failures" step="1"><br>
        Absences (0-20): <input type="number" name="absences" step="1"><br>
        G1 (0-20): <input type="number" name="G1" step="1"><br>
        G2 (0-20): <input type="number" name="G2" step="1"><br>
        <input type="submit" value="Predict">
    </form>
    {% if grade %}
    <h3>Predicted Grade: {{ grade }}</h3>
    <h3>Level: {{ level }}</h3>
    <h3>Recommended Content: {{ content }}</h3>
    {% endif %}
</body>
</html>
"""

# Flask route
@app.route("/", methods=["GET", "POST"])
def predict():
    grade = level = content = None
    if request.method == "POST":
        # Get input values from form
        studytime = float(request.form['studytime'])
        failures = float(request.form['failures'])
        absences = float(request.form['absences'])
        G1 = float(request.form['G1'])
        G2 = float(request.form['G2'])

        # Prepare input DataFrame
        X_input = pd.DataFrame([[studytime, failures, absences, G1, G2]],
                               columns=['studytime','failures','absences','G1','G2'])

        # Predict grade
        grade = model.predict(X_input)[0]
        level = categorize_student(grade)
        content = recommend_content(level)

    return render_template_string(html_template, grade=grade, level=level, content=content)

# Run the Flask app
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            